In [1]:
import ollama

response = ollama.chat(model='llama2', messages=[
    {"role": "system", "content": "You are an AI that helps analyze app reviews."},
    {"role": "user", "content": "Summarize these app reviews: 'The app crashes often.', 'I love the UI!', 'Login issues persist'."}
])

print(response['message']['content'])


Here is a summary of the app reviews you provided:

* The app crashes often, which may be frustrating for users who rely on it for important tasks or functions.
* One user has a positive opinion of the app's User Interface (UI), possibly finding it visually appealing or easy to navigate.
* Another user experiences persistent login issues, which could indicate problems with the app's authentication mechanisms or server connectivity.


In [2]:
import pandas as pd

df = pd.read_csv("cleaned_reviews.csv")
reviews = df["review"].tolist()

# Process first 10 reviews with LLaMA
response = ollama.chat(model='llama2', messages=[
    {"role": "system", "content": "Summarize app reviews for product improvement."},
    {"role": "user", "content": "Summarize these reviews:\n" + "\n".join(reviews[:10])}
])

print(response['message']['content'])


Based on the reviews you provided, here are some common issues and suggestions for improvement:

1. Crashing and force closing: Many users have reported that the app crashes or force closes frequently, which can be frustrating and disruptive. To improve this issue, the developers could focus on optimizing the app's performance and stability.
2. Uninstallation difficulties: Some users have struggled to uninstall the app, even after multiple attempts. This could be due to a bug or limitation in the app itself, or potentially due to interference from other apps or system settings. The developers may need to investigate and address any underlying issues.
3. Virus concerns: A few users have reported that the app has viruses or malware, which can harm their phones or steal personal data. To address this issue, the developers could implement stronger security measures, such as virus scanning or encryption.
4. Battery drain: Some users have noticed that the app drains their battery unexpectedl

In [ ]:
def analyze_sentiment(review):
    response = ollama.chat(model='llama2', messages=[
        {"role": "system", "content": "Classify the sentiment as Positive, Negative, or Neutral."},
        {"role": "user", "content": f"Review: '{review}'"}
    ])
    return response['message']['content']

df['sentiment'] = df['review'].apply(analyze_sentiment)
print(df.head())

In [ ]:
# Save the new dataset
new_file_path = "analyze_sentiment.csv"
df.to_csv(new_file_path, index=False, encoding="utf-8")

In [ ]:
def classify_with_sarcasm(review):
    prompt = (
        "Classify the sentiment of the following app review as Positive, Negative, or Neutral. "
        "Be careful to detect sarcasm or irony. Do not rely only on surface-level positive words.\n\n"
        f"Review: \"{review}\""
    )
    response = ollama.chat(
        model='llama2',
        messages=[
            {"role": "system", "content": "You are a language model trained to analyze app reviews."},
            {"role": "user", "content": prompt}
        ]
    )
    return response['message']['content'].strip()

sub_df = df.head(10)
sub_df['new_sentiment'] = sub_df['review'].apply(classify_with_sarcasm)
print(sub_df.head())

In [ ]:
# Save the new dataset
new_file_path = "analyze_sentiment.csv"
sub_df.to_csv(new_file_path, index=False, encoding="utf-8")

In [1]:
import ollama
import pandas as pd

df = pd.read_csv("cleaned_reviews.csv")
reviews = df["review"].tolist()

def classify_with_sarcasm(review):
    prompt = (
        "Classify the sentiment of the following app review as Positive, Negative, Neutral, or Unsure. "
        "Be careful to detect sarcasm, irony, or misleading praise. Focus on the intent of the entire message.\n\n"
        f"Review: \"{review}\""
    )
    response = ollama.chat(
        model='llama2',
        messages=[
            {"role": "system", "content": "You are a sentiment analysis model trained to detect subtle emotion and sarcasm."},
            {"role": "user", "content": prompt}
        ]
    )
    return response['message']['content'].strip()

# Process the next 4 batches of 10 rows (rows 10 to 50)
for start in range(0, 50, 10):
    end = start + 10
    print(f"Processing rows {start} to {end-1}")
    df.loc[start:end-1, 'sarcasm_sentiment'] = df.loc[start:end-1, 'review'].apply(classify_with_sarcasm)

Processing rows 0 to 9
Processing rows 10 to 19
Processing rows 20 to 29
Processing rows 30 to 39
Processing rows 40 to 49


In [2]:
subset_df = df.head(50).copy()

output_path = 'processed_reviews.xlsx'
subset_df[['review', 'sarcasm_sentiment']].to_excel(output_path, index=False)

print(f"Done! Your processed file is saved as: {output_path}")

Done! Your processed file is saved as: processed_reviews.xlsx


In [3]:
import ollama

# Your review
sarcasm_traps = [
    "Glad the app finally loads... only took 10 minutes and three crashes.",
    "Wow, such a helpful update — none of my settings work now.",
    "If stability means freezing every time I open it, then yes, very stable.",
    "The app didn’t do what I wanted, but at least the icon looks nice.",
    "It's amazing how it manages to get worse with every single update."
]

review_text = "Me: Hi mom, I suggest you to change your job if you are facing issues with the current job. Mom: Did you start applying for companies or not?"

# Instructional-style prompt (ChatGPT-style tuning)
for text in sarcasm_traps:
    prompt = (
        "You are a sentiment analysis model. Classify the sentiment of the following app review as Positive, Negative, Neutral, or Unsure.\n"
        "Pay close attention to sarcasm, irony, and emotional contradictions. If the review includes frustration masked as humor or false praise, mark it as Negative.\n\n"
        f"Review: \"{text}\""
    )

    # Send to LLaMA
    response = ollama.chat(
        model='llama2',
        messages=[
            {"role": "system", "content": "You are an emotionally aware sentiment classifier."},
            {"role": "user", "content": prompt}
        ]
    )
    
    # Output
    print("Sentiment:", response['message']['content'].strip())

Sentiment: Based on the review provided, I would classify the sentiment as Neutral. Here's why:

The reviewer mentions that they are glad the app finally loads after several minutes of waiting and multiple crashes. On the surface, this could be interpreted as a Positive sentiment since the reviewer is expressing relief that the app is working properly. However, there are some signs of frustration and irritation in the review that suggest a more neutral or even Negative sentiment:

1. The use of sarcasm: The reviewer says they are "glad" the app finally loads, which could be seen as a sarcastic comment given the multiple crashes they experienced. This tone suggests frustration rather than genuine enthusiasm.
2. The mention of time: The reviewer notes that it took 10 minutes for the app to load, which is a significant amount of time to spend waiting for an app to launch. This could indicate frustration or impatience with the app's performance.
3. The mention of crashes: The reviewer ment